In [1]:
!pip install tensorflow numpy matplotlib scikit-learn pillow pandas scikeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 32.7 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 1.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 26.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 124.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 78.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 22.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 41.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 41.5 MB/s eta 0:00:00

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout

from tensorflow.keras.applications import EfficientNetB0

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import numpy as np

2024-06-19 18:47:49.425239: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-19 18:47:50.663017: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
rain_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [6]:
# Define the data augmentation for training data
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation data should not be augmented, only rescaled
test_datagen = ImageDataGenerator(rescale=1./255)

# Create the data generators
train_generator = train_datagen.flow_from_directory(
    'images/training',  # Path to your training data
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    'images/testing',  # Path to your validation data
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 2393 images belonging to 3 classes.
Found 597 images belonging to 3 classes.


In [7]:
def create_baseline_model(input_shape=(224, 224, 3), num_classes=3, learning_rate=0.001, l2_reg=0.001, activation='relu', optimizer='adam'):
    model = Sequential([
        Conv2D(32, (3, 3), activation=activation, input_shape=input_shape, kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation=activation, kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation=activation, kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Conv2D(256, (3, 3), activation=activation, kernel_regularizer=l2(l2_reg)),
        BatchNormalization(),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(256, activation=activation, kernel_regularizer=l2(l2_reg)),
        Dropout(0.5),
        Dense(128, activation=activation, kernel_regularizer=l2(l2_reg)),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])
    
    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer == 'adagrad':
        opt = tf.keras.optimizers.Adagrad(learning_rate=learning_rate)
    
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model


In [8]:
model = KerasClassifier(model=create_baseline_model, input_shape=(224, 224, 3), num_classes=3)


In [9]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, min_delta=0.05, mode='max', verbose=1)


In [10]:
param_grid = {
    'model__learning_rate': [0.0001, 0.01],
    'model__l2_reg': [0.0001, 0.01],
    'model__activation': ['relu', 'tanh'],
    'model__optimizer': ['adam', 'sgd'],
    'fit__epochs': [1],
    'fit__batch_size': [32]
}


In [ ]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=2, n_jobs=1)
grid_search_result = grid_search.fit(train_generator, validation_data=test_generator)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END fit__batch_size=32, fit__epochs=1, model__activation=relu, model__l2_reg=0.0001, model__learning_rate=0.0001, model__optimizer=adam; total time=  49.3s
[CV] END fit__batch_size=32, fit__epochs=1, model__activation=relu, model__l2_reg=0.0001, model__learning_rate=0.0001, model__optimizer=adam; total time=  46.5s
[CV] END fit__batch_size=32, fit__epochs=1, model__activation=relu, model__l2_reg=0.0001, model__learning_rate=0.0001, model__optimizer=adam; total time=  46.2s
[CV] END fit__batch_size=32, fit__epochs=1, model__activation=relu, model__l2_reg=0.0001, model__learning_rate=0.0001, model__optimizer=sgd; total time=  46.3s
[CV] END fit__batch_size=32, fit__epochs=1, model__activation=relu, model__l2_reg=0.0001, model__learning_rate=0.0001, model__optimizer=sgd; total time=  46.0s
[CV] END fit__batch_size=32, fit__epochs=1, model__activation=relu, model__l2_reg=0.0001, model__learning_rate=0.0001, model__optimize

In [ ]:
print(f"Best: {grid_search_result.best_score_} using {grid_search_result.best_params_}")

In [ ]:
best_params = grid_search_result.best_params_

In [ ]:
final_model = create_baseline_model(
    input_shape=(224, 224, 3),
    num_classes=3,
    learning_rate=best_params['model__learning_rate'],
    l2_reg=best_params['model__l2_reg'],
    activation=best_params['model__activation'],
    optimizer=best_params['model__optimizer']
)

history_final = final_model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=best_params['fit__epochs'],
    batch_size=best_params['fit__batch_size'],
    callbacks=[reduce_lr, early_stopping, lr_scheduler]
)


In [ ]:
def create_resnet50_model(input_shape=(224, 224, 3), num_classes=3, learning_rate=0.001, activation='relu', optimizer='adam'):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation=activation)(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer == 'adagrad':
        opt = tf.keras.optimizers.Adagrad(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = KerasClassifier(model=create_resnet50_model, input_shape=(224, 224, 3), num_classes=3)

In [ ]:
param_grid = {
    'model__learning_rate': [0.0001, 0.001, 0.01],
    'model__activation': ['relu', 'tanh', 'sigmoid'],
    'model__optimizer': ['adam', 'sgd', 'rmsprop', 'adagrad'],
    'fit__epochs': [1],
    'fit__batch_size': [32, 64]
}


In [ ]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=2, n_jobs=1)
grid_search_result = grid_search.fit(train_generator, validation_data=test_generator)

In [ ]:
print(f"Best: {grid_search_result.best_score_} using {grid_search_result.best_params_}")

In [ ]:
best_params = grid_search_result.best_params_


In [ ]:
final_model = create_resnet50_model(
    input_shape=(224, 224, 3),
    num_classes=3,
    learning_rate=best_params['model__learning_rate'],
    activation=best_params['model__activation'],
    optimizer=best_params['model__optimizer']
)

history_final = final_model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=best_params['fit__epochs'],
    batch_size=best_params['fit__batch_size'],
    callbacks=[reduce_lr, early_stopping, lr_scheduler]
)


In [ ]:
def create_efficientnet_model(input_shape=(224, 224, 3), num_classes=3, learning_rate=0.001, activation='relu', optimizer='adam'):
    base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(128, activation=activation)(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    for layer in base_model.layers:
        layer.trainable = False
    if optimizer == 'adam':
        opt = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'sgd':
        opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = tf.keras.optimizers.RMSprop(learning_rate=learning_rate)
    elif optimizer == 'adagrad':
        opt = tf.keras.optimizers.Adagrad(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = KerasClassifier(model=create_efficientnet_model, input_shape=(224, 224, 3), num_classes=3)

In [ ]:
param_grid = {
    'model__learning_rate': [0.0001, 0.001, 0.01],
    'model__activation': ['relu', 'tanh', 'sigmoid'],
    'model__optimizer': ['adam', 'sgd', 'rmsprop', 'adagrad'],
    'fit__epochs': [1],
    'fit__batch_size': [32, 64]
}


In [ ]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='accuracy', cv=3, verbose=2, n_jobs=1)
grid_search_result = grid_search.fit(train_generator, validation_data=test_generator)


In [ ]:
print(f"Best: {grid_search_result.best_score_} using {grid_search_result.best_params_}")

In [ ]:
best_params = grid_search_result.best_params_

In [ ]:
final_model = create_efficientnet_model(
    input_shape=(224, 224, 3),
    num_classes=3,
    learning_rate=best_params['model__learning_rate'],
    activation=best_params['model__activation'],
    optimizer=best_params['model__optimizer']
)

history_final = final_model.fit(
    train_generator,
    validation_data=test_generator,
    epochs=best_params['fit__epochs'],
    batch_size=best_params['fit__batch_size'],
    callbacks=[reduce_lr, early_stopping, lr_scheduler]
)


In [ ]:
history_baseline = baseline_model.fit(
    train_generator, 
    validation_data=test_generator, 
    epochs=30, 
    callbacks=[reduce_lr, early_stopping, lr_scheduler]
)
history_resnet50 = resnet50_model.fit(
    train_generator, 
    validation_data=test_generator, 
    epochs=30, 
    callbacks=[reduce_lr, early_stopping, lr_scheduler]
)
history_efficientnet = efficientnet_model.fit(
    train_generator, 
    validation_data=test_generator, 
    epochs=30, 
    callbacks=[reduce_lr, early_stopping, lr_scheduler]
)

In [ ]:
def ensemble_predict(models, test_generator):
    X = np.concatenate([model.predict(test_generator) for model in models], axis=1)
    y = test_generator.classes
    
    meta_model = LogisticRegression(max_iter=1000)
    meta_model.fit(X, y)
    
    X_test = np.concatenate([model.predict(test_generator) for model in models], axis=1)
    y_pred = meta_model.predict(X_test)
    
    return y_pred

In [ ]:
models = [final_baseline_model, resnet50_model, efficientnet_model]
ensemble_predictions = ensemble_predict(models, test_generator)

In [ ]:
correct_labels = test_generator.classes
ensemble_accuracy = accuracy_score(correct_labels, ensemble_predictions)
print(f'Ensemble Accuracy: {ensemble_accuracy:.2f}')